In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import RFE
from scipy.stats import zscore

In [3]:
# Load dataset
data = pd.read_csv("Housing - Housing.csv")  # Replace with actual file path

In [4]:
print(data.columns)  # Check available columns
data.columns = data.columns.str.strip()  # Remove any leading/trailing spaces

Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus'],
      dtype='object')


In [5]:
# Convert categorical variables to numerical
categorical_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
data[categorical_cols] = data[categorical_cols].replace({'yes': 1, 'no': 0})

# One-hot encode 'furnishingstatus'
data = pd.get_dummies(data, columns=['furnishingstatus'], drop_first=True)

# Remove outliers using Z-score (threshold = 3)
data = data[(np.abs(zscore(data.select_dtypes(include=[np.number]))) < 3).all(axis=1)]

# Define features and target variable
X = data.drop(columns=['price'])
y = np.log1p(data['price'])  # Apply log transformation to stabilize variance

# Split data into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Feature Selection using RFE
model = LinearRegression()
rfe = RFE(model, n_features_to_select=10)  # Adjust number of features
X_train_selected = rfe.fit_transform(X_train_scaled, y_train)
X_test_selected = rfe.transform(X_test_scaled)

# Train and evaluate different models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1)
}

for name, model in models.items():
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"{name}:")
    print("  Mean Absolute Error (MAE):", mae)
    print("  Root Mean Squared Error (RMSE):", rmse)
    print("  R-squared (R2):", r2)
    print("-" * 40)


Linear Regression:
  Mean Absolute Error (MAE): 0.17443304130091664
  Root Mean Squared Error (RMSE): 0.21999930194879713
  R-squared (R2): 0.6869028360980141
----------------------------------------
Ridge Regression:
  Mean Absolute Error (MAE): 0.1744505071732555
  Root Mean Squared Error (RMSE): 0.22007058735189827
  R-squared (R2): 0.6866999002400717
----------------------------------------
Lasso Regression:
  Mean Absolute Error (MAE): 0.2583416651286193
  Root Mean Squared Error (RMSE): 0.3233854900158352
  R-squared (R2): 0.32348475552366196
----------------------------------------


C:\Users\HP\AppData\Local\Temp\ipykernel_10052\2345179479.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[categorical_cols] = data[categorical_cols].replace({'yes': 1, 'no': 0})
